In [15]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [16]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '5827d66bb4f301d3217934efd0d515cc', 'Date': 'Tue, 20 Aug 2024 17:15:47 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [17]:
import json
data = json.load(open('reviews.json'))
data['reviews']


[{'professor': 'Dr. Emma Thompson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures and fair exams. Dr. Thompson really knows how to make cell biology interesting!'},
 {'professor': 'Prof. David Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant teacher! His algorithms course was challenging but incredibly rewarding.'},
 {'professor': 'Dr. Sarah Patel',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Decent course content, but lectures can be a bit dry. Appreciate the real-world examples though.'},
 {'professor': "Prof. Michael O'Brien",
  'subject': 'History',
  'stars': 5,
  'review': "Prof. O'Brien's passion for medieval history is contagious. Best class I've taken so far!"},
 {'professor': 'Dr. Lisa Yamamoto',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Quantum mechanics made (somewhat) understandable. Dr. Yamamoto is patient and explains complex concepts well.'},
 {'professor': 'Prof. Robert Johnson',
  'subject': 'Mathem

In [19]:
process_data = []
client = OpenAI()

for review in data['reviews']:
  response = client.embeddings.create(
    input=review['review'],
    model="text-embedding-3-small",
  )
  embedding = response.data[0].embedding
  process_data.append({
    "values": embedding,
    "id": review['professor'],
    "metadata": {
      "review":review['review'],
      "subject":review['subject'],
      "stars": review['stars']


    }}

  )

In [20]:
process_data[0]

{'values': [0.017012231,
  -0.009396647,
  0.007210238,
  0.05443914,
  0.006319706,
  -0.012086668,
  0.042942066,
  0.009218541,
  -0.006731193,
  0.01896526,
  0.03382793,
  0.005214218,
  -0.008573673,
  -0.012313907,
  0.003540632,
  -0.024197903,
  0.0040595974,
  0.00031302965,
  0.037905954,
  0.03579324,
  0.036186304,
  -0.032575045,
  0.025770152,
  -0.020881439,
  -0.06863852,
  -0.0699651,
  0.024517266,
  -0.0036911014,
  -0.020955138,
  -0.0060863253,
  0.06239865,
  -0.02906205,
  0.0043697483,
  -0.024689231,
  -0.034073595,
  0.058369763,
  0.01650862,
  0.04114872,
  -0.0018440153,
  0.0069277245,
  0.0518351,
  -0.010152064,
  -0.031813487,
  0.022023777,
  -0.003075406,
  -0.023239812,
  -0.007984079,
  -0.044416048,
  0.028349623,
  0.039379936,
  0.0021434182,
  0.032206547,
  0.029356847,
  -0.029577944,
  -0.07414139,
  0.039109707,
  -0.025094576,
  0.051687703,
  -0.04733945,
  -0.028005693,
  0.037365492,
  -0.0076769996,
  0.02933228,
  -0.0074620433,
  -0.

In [22]:
index = pc.Index("rag")
index.upsert(
  vectors = process_data,
  namespace="ns1"

)

{'upserted_count': 20}

In [23]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}